<a href="https://colab.research.google.com/github/Jkimoon/2025S/blob/main/Builty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
import requests
import time

API_KEY = "kESYeXsR.gTSS3YhtRpF28cVum6jkv2sHYxaq15ItmMShbDXgC7UOOqBXtqA9h6Lx"

def make_get_request(url, params = None, attempt_count = 1, max_attempts = 5):
    """call API using GET request, including retry logic"""
    headers = {'X-API-KEY': API_KEY,
               'Content-Type': 'application/json'}

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 429 and attempt_count <= max_attempts:
        sleep_time = (2 ** attempt_count) + random.uniform(0, 2)
        print(f'Ratelimit reached, retrying after {sleep_time:.1f} seconds')
        time.sleep(sleep_time)
        return make_get_request(url=url, params=params, attempt_count=attempt_count+1)

    if response.status_code == 400:
        raise Exception(f'Request failed with {response.status_code}')

    if not (response.status_code >= 200 and response.status_code < 300):
        raise Exception(f'Request failed with {response.status_code} and {response.json()}')

    return response



In [3]:
# set API URL
PRODUCT_API_PATH = "https://app.deweydata.io/external-api/v3/products/711cbce0-d879-4cf8-8027-0b865009b798/files"

# loop through all API result pages, keeping track of number of downloaded files
page = 1
download_count = 0
while True:
    # get results from API endpoint, using API key for authentication, for a specific page
    results = make_get_request(
        url=PRODUCT_API_PATH,
        params={'page': page}
    )
    response_json = results.json()

    # for each result page, loop through download links and save to your computer
    for link_data in response_json['download_links']:
        print(f"Downloading file {link_data['file_name']}...")
        data = make_get_request(link_data['link'])
        with open(link_data['file_name'], 'wb') as file:
            file.write(data.content)
        download_count += 1

    # only continue if there are more result pages to process
    total_pages = response_json['total_pages']
    if page >= total_pages:
        break
    page += 1

print(f"Successfully downloaded {download_count} files.")

OSError: [Errno 28] No space left on device